In [ ]:
# Exercise 1

import torch
import torch.nn as nn
import math

class LoRALayer(nn.Module):
    def __init__(self, 
                 in_features,    # Dimension d'entrée
                 out_features,   # Dimension de sortie
                 rank=4,         # Rang de la décomposition
                 alpha=1.0):     # Facteur d'échelle
        super().__init__()
        
        # Initialisation des dimensions
        self.in_features = in_features
        self.out_features = out_features
        self.rank = rank
        self.alpha = alpha
        self.scaling = alpha / rank
        
        # Initialisation des matrices A et B
        # A : matrice de dimension (rank x in_features)
        # B : matrice de dimension (out_features x rank)
        self.A = nn.Parameter(torch.zeros(rank, in_features))
        self.B = nn.Parameter(torch.zeros(out_features, rank))
        
        # Initialisation des poids avec une distribution normale
        nn.init.kaiming_uniform_(self.A, a=math.sqrt(5))
        nn.init.zeros_(self.B)  # Initialisation à zéro pour B
        
    def forward(self, x):
        # x : tensor d'entrée de dimension (batch_size x in_features)
        # Calcul de la transformation LoRA : x → (B·A)x
        # 1. Multiplication avec A
        first_proj = torch.matmul(x, self.A.T)  # (batch_size x rank)
        # 2. Multiplication avec B et application du scaling
        lora_output = torch.matmul(first_proj, self.B.T) * self.scaling  # (batch_size x out_features)
        return lora_output

# Test de la classe
def test_lora_layer():
    # Paramètres de test
    batch_size = 32
    in_features = 768   # Dimension typique d'un modèle transformeur
    out_features = 512
    rank = 4
    
    # Création de la couche LoRA
    lora = LoRALayer(in_features, out_features, rank)
    
    # Création d'un tensor d'entrée aléatoire
    x = torch.randn(batch_size, in_features)
    
    # Forward pass
    output = lora(x)
    
    # Vérifications
    print(f"Dimension d'entrée: {x.shape}")
    print(f"Dimension de sortie: {output.shape}")
    print(f"Nombre de paramètres entraînables: {sum(p.numel() for p in lora.parameters())}")

# Exécution du test
test_lora_layer()

Dimension d'entrée: torch.Size([32, 768])
Dimension de sortie: torch.Size([32, 512])
Nombre de paramètres entraînables: 5120


In [ ]:
# Exercise 2

class LinearWithLoRA(nn.Module):
    def __init__(self,
                 in_features,
                 out_features,
                 rank=4,
                 alpha=1.0,
                 bias=True):
        super().__init__()
        
        self.linear = nn.Linear(in_features, out_features, bias=bias)
        
        self.lora = LoRALayer(
            in_features=in_features,
            out_features=out_features,
            rank=rank,
            alpha=alpha
        )
        
        # Freeze les paramètres de la couche linéaire
        for param in self.linear.parameters():
            param.requires_grad = False
            
    def forward(self, x):
        return self.linear(x) + self.lora(x)

# Fonction de test
def test_linear_with_lora():
    batch_size = 32
    in_features = 768  # Comme dans votre projet de classification de mobiles
    out_features = 512
    rank = 4
    
    layer = LinearWithLoRA(
        in_features=in_features,
        out_features=out_features,
        rank=rank
    )
    
    x = torch.randn(batch_size, in_features)
    output = layer(x)
    
    print("Test de LinearWithLoRA :")
    print(f"Dimension d'entrée: {x.shape}")
    print(f"Dimension de sortie: {output.shape}")
    print("\nParamètres entraînables:")
    trainable_params = sum(p.numel() for p in layer.parameters() if p.requires_grad)
    print(f"Nombre total de paramètres entraînables: {trainable_params}")
    print(f"Dont LoRA: {sum(p.numel() for p in layer.lora.parameters())}")

# Exécution du test
if __name__ == "__main__":
    test_linear_with_lora()


Test de LinearWithLoRA :
Dimension d'entrée: torch.Size([32, 768])
Dimension de sortie: torch.Size([32, 512])

Paramètres entraînables:
Nombre total de paramètres entraînables: 5120
Dont LoRA: 5120


In [8]:
# Exercise 3

import torch
import torch.nn as nn

class SimpleNetworkWithLoRA(nn.Module):
    def __init__(self, 
                 input_dim=768,    # Dimension similaire à vos exercices RAG
                 hidden_dim=512,
                 output_dim=256,
                 rank=4):
        super().__init__()
        
        # Première couche avec LoRA
        self.layer1 = LinearWithLoRA(
            in_features=input_dim,
            out_features=hidden_dim,
            rank=rank
        )
        
        # Activation ReLU
        self.relu = nn.ReLU()
        
        # Couche de sortie standard
        self.layer2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x

def test_network():
    # Paramètres de test
    batch_size = 32
    input_dim = 768  # Dimension similaire à vos embeddings RAG
    hidden_dim = 512
    output_dim = 256
    
    # Création des réseaux (standard et avec LoRA)
    network_standard = nn.Sequential(
        nn.Linear(input_dim, hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim, output_dim)
    )
    
    network_lora = SimpleNetworkWithLoRA(
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        output_dim=output_dim
    )
    
    # Données de test
    x = torch.randn(batch_size, input_dim)
    
    # Test des deux réseaux
    with torch.no_grad():
        output_standard = network_standard(x)
        output_lora = network_lora(x)
    
    # Affichage des résultats
    print("Test des réseaux neuronaux :")
    print(f"Dimension d'entrée: {x.shape}")
    print(f"Dimension de sortie (standard): {output_standard.shape}")
    print(f"Dimension de sortie (LoRA): {output_lora.shape}")
    
    # Analyse des paramètres entraînables
    params_standard = sum(p.numel() for p in network_standard.parameters() if p.requires_grad)
    params_lora = sum(p.numel() for p in network_lora.parameters() if p.requires_grad)
    
    print("\nComparaison des paramètres :")
    print(f"Paramètres entraînables (standard): {params_standard}")
    print(f"Paramètres entraînables (LoRA): {params_lora}")
    print(f"Réduction des paramètres: {(1 - params_lora/params_standard)*100:.2f}%")

# Exécution du test
if __name__ == "__main__":
    test_network()


Test des réseaux neuronaux :
Dimension d'entrée: torch.Size([32, 768])
Dimension de sortie (standard): torch.Size([32, 256])
Dimension de sortie (LoRA): torch.Size([32, 256])

Comparaison des paramètres :
Paramètres entraînables (standard): 525056
Paramètres entraînables (LoRA): 136448
Réduction des paramètres: 74.01%


In [ ]:
# Exercise 4

import torch
import torch.nn as nn
import math

class LinearWithLoRAMerged(nn.Module):
    def __init__(self,
                 in_features,
                 out_features,
                 rank=4,
                 alpha=1.0,
                 bias=True):
        super().__init__()
        
        # Couche linéaire standard
        self.linear = nn.Linear(in_features, out_features, bias=bias)
        
        # Matrices LoRA
        self.lora_A = nn.Parameter(torch.zeros(rank, in_features))
        self.lora_B = nn.Parameter(torch.zeros(out_features, rank))
        self.scaling = alpha / rank
        
        # Initialisation des poids LoRA
        nn.init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))
        nn.init.zeros_(self.lora_B)
        
        # Matrice des poids fusionnés (initialement None)
        self.merged_weights = None
        
    def merge_weights(self):
        """Fusionne les poids LoRA avec la matrice de poids principale"""
        if self.merged_weights is None:
            # Calcul de la matrice LoRA
            lora_matrix = torch.matmul(self.lora_B, self.lora_A) * self.scaling
            
            # Fusion avec les poids originaux
            self.merged_weights = self.linear.weight.data.clone()
            self.merged_weights += lora_matrix
            
    def unmerge_weights(self):
        """Restaure les poids originaux"""
        self.merged_weights = None
            
    def forward(self, x):
        if self.merged_weights is not None:
            # Utilisation des poids fusionnés
            return torch.nn.functional.linear(x, self.merged_weights, self.linear.bias)
        else:
            # Calcul séparé (comme dans LinearWithLoRA)
            linear_output = self.linear(x)
            lora_output = torch.matmul(x, self.lora_A.T)
            lora_output = torch.matmul(lora_output, self.lora_B.T) * self.scaling
            return linear_output + lora_output

def test_lora_merged():
    # Paramètres de test
    batch_size = 32
    in_features = 768  # Comme dans vos exercices RAG
    out_features = 512
    rank = 4
    
    # Création des modèles pour comparaison
    model_standard = LinearWithLoRA(
        in_features=in_features,
        out_features=out_features,
        rank=rank
    )
    
    model_merged = LinearWithLoRAMerged(
        in_features=in_features,
        out_features=out_features,
        rank=rank
    )
    
    # Copie des poids pour assurer l'équivalence
    model_merged.linear.weight.data = model_standard.linear.weight.data.clone()
    model_merged.linear.bias.data = model_standard.linear.bias.data.clone()
    model_merged.lora_A.data = model_standard.lora.A.data.clone()
    model_merged.lora_B.data = model_standard.lora.B.data.clone()
    
    # Données de test
    x = torch.randn(batch_size, in_features)
    
    # Test des sorties
    with torch.no_grad():
        output_standard = model_standard(x)
        
        # Test sans fusion
        output_merged_before = model_merged(x)
        
        # Test avec fusion
        model_merged.merge_weights()
        output_merged_after = model_merged(x)
    
    # Vérification de l'équivalence
    print("Test d'équivalence des sorties :")
    print(f"Différence max avant fusion: {(output_standard - output_merged_before).abs().max().item()}")
    print(f"Différence max après fusion: {(output_standard - output_merged_after).abs().max().item()}")
    
    # Comparaison des performances
    import time
    
    def measure_time(model, x, n_runs=1000):
        start = time.time()
        for _ in range(n_runs):
            with torch.no_grad():
                _ = model(x)
        return (time.time() - start) / n_runs
    
    time_standard = measure_time(model_standard, x)
    time_merged = measure_time(model_merged, x)
    
    print("\nComparaison des performances :")
    print(f"Temps moyen (standard): {time_standard*1000:.3f} ms")
    print(f"Temps moyen (fusionné): {time_merged*1000:.3f} ms")
    print(f"Accélération: {time_standard/time_merged:.2f}x")

# Exécution du test
if __name__ == "__main__":
    test_lora_merged()


Test d'équivalence des sorties :
Différence max avant fusion: 0.0
Différence max après fusion: 0.0

Comparaison des performances :
Temps moyen (standard): 1.926 ms
Temps moyen (fusionné): 0.873 ms
Accélération: 2.21x


In [ ]:
# Exercise 5

import torch
import torch.nn as nn

class MLPWithLoRA(nn.Module):
    def __init__(self,
                 input_dim=768,    # Dimension similaire à vos embeddings RAG
                 hidden_dims=[512, 256],
                 output_dim=128,
                 rank=4,
                 dropout=0.1):
        super().__init__()
        
        self.layers = nn.ModuleList()
        
        # Construction des couches
        dims = [input_dim] + hidden_dims + [output_dim]
        for i in range(len(dims)-1):
            # Ajout de la couche LinearWithLoRAMerged
            self.layers.append(LinearWithLoRAMerged(
                in_features=dims[i],
                out_features=dims[i+1],
                rank=rank
            ))
            
            # Ajout de l'activation et dropout (sauf pour la dernière couche)
            if i < len(dims)-2:
                self.layers.append(nn.ReLU())
                self.layers.append(nn.Dropout(dropout))
    
    def merge_lora_weights(self):
        """Fusionne les poids LoRA dans toutes les couches"""
        for layer in self.layers:
            if isinstance(layer, LinearWithLoRAMerged):
                layer.merge_weights()
    
    def unmerge_lora_weights(self):
        """Défusionne les poids LoRA dans toutes les couches"""
        for layer in self.layers:
            if isinstance(layer, LinearWithLoRAMerged):
                layer.unmerge_weights()
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

def test_mlp_with_lora():
    # Paramètres de test
    batch_size = 32
    input_dim = 768  # Dimension type pour vos embeddings RAG
    hidden_dims = [512, 256]
    output_dim = 128
    rank = 4
    
    # Création du modèle
    model = MLPWithLoRA(
        input_dim=input_dim,
        hidden_dims=hidden_dims,
        output_dim=output_dim,
        rank=rank
    )
    
    # Affichage de l'architecture
    print("Architecture du MLP avec LoRA :")
    for i, layer in enumerate(model.layers):
        print(f"Couche {i}: {layer.__class__.__name__}")
    
    # Test avec données
    x = torch.randn(batch_size, input_dim)
    
    # Test avant fusion
    print("\nTest de forward pass :")
    output_before = model(x)
    print(f"Dimension d'entrée: {x.shape}")
    print(f"Dimension de sortie: {output_before.shape}")
    
    # Test des performances avec et sans fusion
    import time
    
    def measure_time(model, x, n_runs=1000):
        start = time.time()
        for _ in range(n_runs):
            with torch.no_grad():
                _ = model(x)
        return (time.time() - start) / n_runs
    
    # Test sans fusion
    time_before = measure_time(model, x)
    
    # Test avec fusion
    model.merge_lora_weights()
    time_after = measure_time(model, x)
    
    print("\nComparaison des performances :")
    print(f"Temps moyen (sans fusion): {time_before*1000:.3f} ms")
    print(f"Temps moyen (avec fusion): {time_after*1000:.3f} ms")
    print(f"Accélération: {time_before/time_after:.2f}x")
    
    # Calcul des paramètres entraînables
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"\nNombre total de paramètres entraînables: {trainable_params}")

# Exécution du test
if __name__ == "__main__":
    test_mlp_with_lora()


Architecture du MLP avec LoRA :
Couche 0: LinearWithLoRAMerged
Couche 1: ReLU
Couche 2: Dropout
Couche 3: LinearWithLoRAMerged
Couche 4: ReLU
Couche 5: Dropout
Couche 6: LinearWithLoRAMerged

Test de forward pass :
Dimension d'entrée: torch.Size([32, 768])
Dimension de sortie: torch.Size([32, 128])

Comparaison des performances :
Temps moyen (sans fusion): 3.095 ms
Temps moyen (avec fusion): 1.990 ms
Accélération: 1.55x

Nombre total de paramètres entraînables: 567680


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

def freeze_linear_layers(model):
    """Gèle les couches Linear standard et garde les paramètres LoRA entraînables"""
    for module in model.modules():
        if isinstance(module, LinearWithLoRAMerged):
            # Geler les poids de la couche linéaire standard
            module.linear.weight.requires_grad = False
            if module.linear.bias is not None:
                module.linear.bias.requires_grad = False
            
            # Garder les paramètres LoRA entraînables
            module.lora_A.requires_grad = True
            module.lora_B.requires_grad = True

def print_trainable_parameters(model):
    """Affiche les détails des paramètres entraînables"""
    trainable_params = 0
    all_params = 0
    
    for name, param in model.named_parameters():
        all_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
            print(f"Paramètre entraînable: {name}, Shape: {param.shape}")
    
    print(f"\nRatio de paramètres entraînables: {trainable_params/all_params*100:.2f}%")
    print(f"Total paramètres: {all_params}")
    print(f"Paramètres entraînables: {trainable_params}")

def train_and_evaluate(model, train_loader, val_loader, epochs=10):
    """Entraîne et évalue le modèle"""
    criterion = nn.MSELoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
    
    print("Début de l'entraînement...")
    for epoch in range(epochs):
        # Mode entraînement
        model.train()
        train_loss = 0
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            output = model(batch_x)
            loss = criterion(output, batch_y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Mode évaluation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                output = model(batch_x)
                val_loss += criterion(output, batch_y).item()
        
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Train Loss: {train_loss/len(train_loader):.4f}")
        print(f"Val Loss: {val_loss/len(val_loader):.4f}\n")

def test_frozen_mlp():
    # Paramètres adaptés à votre contexte Solweig & Izar
    input_dim = 768  # Dimension type pour vos embeddings
    hidden_dims = [512, 256]
    output_dim = 128
    batch_size = 32
    n_samples = 1000
    
    # Création du modèle
    model = MLPWithLoRA(
        input_dim=input_dim,
        hidden_dims=hidden_dims,
        output_dim=output_dim
    )
    
    # Gel des couches linéaires
    freeze_linear_layers(model)
    
    # Affichage des paramètres entraînables
    print("Analyse des paramètres après gel:")
    print_trainable_parameters(model)
    
    # Création d'un dataset synthétique (simulant vos données de design)
    X = torch.randn(n_samples, input_dim)
    y = torch.randn(n_samples, output_dim)  # Cible synthétique
    
    # Split train/val
    train_size = int(0.8 * n_samples)
    X_train, X_val = X[:train_size], X[train_size:]
    y_train, y_val = y[:train_size], y[train_size:]
    
    # Création des dataloaders
    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
    # Entraînement et évaluation
    train_and_evaluate(model, train_loader, val_loader, epochs=5)
    
    # Test final avec fusion LoRA
    print("\nTest final avec fusion LoRA:")
    model.merge_lora_weights()
    with torch.no_grad():
        test_input = torch.randn(1, input_dim)
        output = model(test_input)
        print(f"Dimension de sortie: {output.shape}")

# Exécution du test
if __name__ == "__main__":
    test_frozen_mlp()


Analyse des paramètres après gel:
Paramètre entraînable: layers.0.lora_A, Shape: torch.Size([4, 768])
Paramètre entraînable: layers.0.lora_B, Shape: torch.Size([512, 4])
Paramètre entraînable: layers.3.lora_A, Shape: torch.Size([4, 512])
Paramètre entraînable: layers.3.lora_B, Shape: torch.Size([256, 4])
Paramètre entraînable: layers.6.lora_A, Shape: torch.Size([4, 256])
Paramètre entraînable: layers.6.lora_B, Shape: torch.Size([128, 4])

Ratio de paramètres entraînables: 1.71%
Total paramètres: 567680
Paramètres entraînables: 9728


/opt/anaconda3/envs/tf_m1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Début de l'entraînement...
Epoch 1/5
Train Loss: 1.0128
Val Loss: 0.9922

Epoch 2/5
Train Loss: 1.0086
Val Loss: 0.9879

Epoch 3/5
Train Loss: 1.0045
Val Loss: 0.9864

Epoch 4/5
Train Loss: 1.0027
Val Loss: 0.9862

Epoch 5/5
Train Loss: 1.0021
Val Loss: 0.9860


Test final avec fusion LoRA:
Dimension de sortie: torch.Size([1, 128])


**Analyse de la convergence** :
Epoch 1 → 5 :
Train Loss: 1.0128 → 1.0021 (↓ 0.0107)
Val Loss: 0.9922 → 0.9860 (↓ 0.0062)


Cette évolution montre :
1. **Une convergence stable** :
   - Diminution régulière de la loss
   - Pas de surapprentissage (validation loss suit la même tendance)

2. **Performance optimisée** :
   - Réduction de ~1% de l'erreur
   - Stabilisation progressive

3. **Dimension de sortie adaptée** :
Dimension de sortie: torch.Size([1, 128])
